In [4]:
import threading
import socket
from tkinter import Tk, PhotoImage, Label, Button, LEFT, Canvas
import pickle
#from PIL import ImageTk, Image, ImageDraw
from os import path
import time
import select
import openpyxl

In [5]:
###################################
#修改！
###################################
map_name = "maps/map009.xlsx"

In [ ]:
while not path.exists(map_name):
    map_name = input("File path: ")
    
wb_obj = openpyxl.load_workbook(map_name)

sheet_obj = wb_obj['map']

size_x = sheet_obj.max_column
size_y = sheet_obj.max_row

grid_size = int(600/max(size_x, size_y))

#################################3

root = Tk()

root.geometry(str(size_x * grid_size + 50) + 'x' + str(size_y * grid_size + 100))

root.title('Monitor')

canvas = Canvas(root, width = size_x * grid_size, height = size_y * grid_size, background = "white")
canvas.pack(ipadx = 10, ipady = 10)

time_label = Label(root, font=("Arial", 12), bg="white")
time_label.pack()

def set_run():
    with open("pause.txt", "w") as f:
        f.write("0")
        
run_button = Button(root, text = "run", command = set_run)
run_button.pack(side = LEFT, padx = 30, pady = 5)

def set_pause():
    with open("pause.txt", "w") as f:
        f.write("1")
        
pause_button = Button(root, text = "pause", command = set_pause)
pause_button.pack(side = LEFT, padx = 30, pady = 5)

def set_step():
    with open("pause.txt", "w") as f:
        f.write("2")
        
pause_button = Button(root, text = "step", command = set_step)
pause_button.pack(side = LEFT, padx = 30, pady = 5)

"""

def set_stop():
    with open("pause.txt", "w") as f:
        f.write("2")
    root.after(3000, set_pause)

stop_button = Button(root, text = "stop", command = set_stop)
stop_button.pack(side = LEFT, padx = 30, pady = 5)

root.after(1, set_pause)

"""

######################################

running = True
Data = None

def recvUDP(*args,**kwargs):
    receiver = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    receiver.bind(('127.0.0.1', 8081))
    receiver.setblocking(False)
    global Data
    while running:
        ready = select.select([receiver], [], [], 1)
        if ready[0]:
            pickel_data, addr = receiver.recvfrom(100000) 
            Data = pickle.loads(pickel_data)
    receiver.close()
    
t1 = threading.Thread(target = recvUDP, args = tuple(), kwargs = dict())

t1.start() 

###################################


def draw_rect(x, y, color):
    canvas.create_rectangle(grid_size * x, grid_size * y, grid_size * x + grid_size - 1, grid_size * y + grid_size - 1,
                             fill = color,
                             outline = None,
                             width = 0)

def refresh_image():
    
    canvas.delete("all")
    
    if Data:
    
        en,ex,cs,ob,l,lu,ld,ll,lr,p,a,t = Data
        
        time_label.config(text=t)
        
        #print(Data)
        
        for record in en:
            draw_rect(record[0][0], size_y - record[0][1] - 1, "yellow")
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1), text=record[1], fill="black", anchor="nw")
            
        for record in ex:
            draw_rect(record[0][0], size_y - record[0][1] - 1, "orange")
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1), text=record[1], fill="black", anchor="nw")
            
        for record in cs:
            draw_rect(record[0][0], size_y - record[0][1] - 1, "blue")
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1), text=record[1], fill="white", anchor="nw")
            
        for record in ob:
            draw_rect(record[0], size_y - record[1] - 1, "gray")
            
        for record in l:
            draw_rect(record[0], size_y - record[1] - 1, "white")

        for record in lu:
            draw_rect(record[0], size_y - record[1] - 1, "white")
            canvas.create_text(grid_size * record[0],grid_size * (size_y - record[1] - 1), text='↑', fill="black", anchor="nw")
            
        for record in ld:
            draw_rect(record[0], size_y - record[1] - 1, "white")
            canvas.create_text(grid_size * record[0],grid_size * (size_y - record[1] - 1), text='↓', fill="black", anchor="nw")
            
        for record in ll:
            draw_rect(record[0], size_y - record[1] - 1, "white")
            canvas.create_text(grid_size * record[0],grid_size * (size_y - record[1] - 1), text='←', fill="black", anchor="nw")
            
        for record in lr:
            draw_rect(record[0], size_y - record[1] - 1, "white")
            canvas.create_text(grid_size * record[0],grid_size * (size_y - record[1] - 1), text='→', fill="black", anchor="nw")
            
        color_list = ['red','orange','blue','purple','black','black']
        for record in p:
            if record[2] == '-1':
                c = 'lightgreen'
            else:
                c = 'green'
            
            draw_rect(record[0][0], size_y - record[0][1] - 1, c)
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1), text=record[1], fill=color_list[record[3]], anchor="nw")
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1)+10, text=record[2], fill=color_list[record[3]], anchor="nw")

        color_list = []
        for i in range(len(a)):
            color_value = int(i/(len(a)-1)*140+35)
            c = "#{:02X}0000".format(color_value)
            color_list.append(c)
            
        for i in range(len(a)):
            record= a[i]
            c = color_list[i]
            draw_rect(record[0][0], size_y - record[0][1] - 1, c)
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1), text=record[1], fill="white", anchor="nw")
            canvas.create_text(grid_size * record[0][0],grid_size * (size_y - record[0][1] - 1)+10, text=record[2], fill="white", anchor="nw")

        # 画网格
        for i in range(size_x):
            canvas.create_line((i + 1) * grid_size - 1, 0, (i + 1) * grid_size - 1, size_y * grid_size - 1, fill = "gray", width = 1)

        for i in range(size_y):
            canvas.create_line(0, (i + 1) * grid_size - 1, size_x * grid_size - 1, (i + 1) * grid_size - 1, fill = "gray", width = 1)
        
    global timer_id
    timer_id = root.after(10, refresh_image)

timer_id = root.after(10, refresh_image)

def on_closing():
    root.after_cancel(timer_id)
    root.destroy()

root.protocol("WM_DELETE_WINDOW", on_closing)

root.mainloop()
running = False
t1.join()